# 05 - Multiscale Detection

A single timeframe can give misleading signals. By running indicators at
**multiple scales** (daily, 3-day, weekly) and measuring their agreement,
we obtain a more robust crash probability.

This notebook:
1. Computes Hill, DFA, and Hurst indicators at daily, 3-day, and weekly scales
2. Computes LPPLS confidence at each scale
3. Measures cross-scale agreement and shows how it strengthens before major crashes

> **DISCLAIMER:** This software is for academic research and educational purposes only.
> It does not constitute financial advice.

## Imports

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

from fatcrash.data.ingest import from_sample
from fatcrash.data.transforms import log_returns, log_prices, time_index, resample_ohlcv
from fatcrash.indicators.lppls_indicator import compute_confidence
from fatcrash.indicators.tail_indicator import (
    rolling_tail_index, rolling_kappa, rolling_dfa, rolling_hurst,
)
from fatcrash.indicators.evt_indicator import rolling_var_es

plt.style.use("seaborn-v0_8-whitegrid")
plt.rcParams["figure.figsize"] = (14, 5)

## 1. Load and resample data

In [ ]:
df = from_sample("btc")

# Create three timeframes
df_daily = df.copy()

df_3day = resample_ohlcv(df, freq="3D")

df_weekly = resample_ohlcv(df, freq="W")

timeframes = {
    "daily": df_daily,
    "3-day": df_3day,
    "weekly": df_weekly,
}

for name, frame in timeframes.items():
    print(f"{name:>8s}: {len(frame)} observations, "
          f"{frame.index[0].date()} to {frame.index[-1].date()}")

## 2. Compute rolling Hill tail index at each scale

In [ ]:
hill_results = {}

# Window sizes adjusted for each timeframe
window_map = {"daily": 500, "3-day": 170, "weekly": 75}

for name, frame in timeframes.items():
    rets = log_returns(frame)
    window = window_map[name]
    if len(rets) < window:
        print(f"{name}: not enough data (need {window}, have {len(rets)})")
        continue
    alpha_arr = rolling_tail_index(rets, window=window)
    # rolling returns are aligned to the end of each window
    dates = frame.index[1:]  # returns are 1 shorter
    hill_results[name] = pd.Series(
        alpha_arr[:len(dates)],
        index=dates[:len(alpha_arr)],
        name=f"alpha_{name}",
    )
    valid = hill_results[name].dropna()
    print(f"{name}: {len(valid)} rolling estimates, "
          f"mean alpha={valid.mean():.2f}")

In [ ]:
fig, axes = plt.subplots(2, 1, figsize=(14, 8), sharex=True)

axes[0].plot(df.index, df["close"], color="steelblue", linewidth=0.8)
axes[0].set_yscale("log")
axes[0].set_ylabel("Price (USD)")
axes[0].set_title("BTC/USD Price")

colors = {"daily": "blue", "3-day": "orange", "weekly": "green"}
for name, series in hill_results.items():
    axes[1].plot(series.index, series.values, color=colors[name],
                 linewidth=0.8, alpha=0.8, label=name)

axes[1].axhline(2, color="red", linestyle="--", alpha=0.5, label="alpha=2")
axes[1].set_ylabel("Tail Index (alpha)")
axes[1].set_title("Rolling Hill Estimator: Multiscale Comparison")
axes[1].set_ylim(0, 6)
axes[1].legend()

plt.tight_layout()
plt.show()

## 3. Rolling DFA and Hurst at each scale

In [ ]:
dfa_results = {}
hurst_results = {}

for name, frame in timeframes.items():
    rets = log_returns(frame)
    window = window_map[name]
    if len(rets) < window:
        continue
    dates = frame.index[1:]

    dfa_arr = rolling_dfa(rets, window=window)
    dfa_results[name] = pd.Series(dfa_arr[:len(dates)], index=dates[:len(dfa_arr)])

    hurst_arr = rolling_hurst(rets, window=window)
    hurst_results[name] = pd.Series(hurst_arr[:len(dates)], index=dates[:len(hurst_arr)])

fig, axes = plt.subplots(2, 1, figsize=(14, 8), sharex=True)

for name, series in dfa_results.items():
    axes[0].plot(series.index, series.values, color=colors[name],
                 linewidth=0.8, alpha=0.8, label=name)
axes[0].axhline(0.5, color="grey", linestyle="--", alpha=0.5)
axes[0].set_ylabel("DFA exponent")
axes[0].set_title("Rolling DFA: Multiscale (>0.5 = persistent)")
axes[0].legend()

for name, series in hurst_results.items():
    axes[1].plot(series.index, series.values, color=colors[name],
                 linewidth=0.8, alpha=0.8, label=name)
axes[1].axhline(0.5, color="grey", linestyle="--", alpha=0.5)
axes[1].set_ylabel("Hurst H")
axes[1].set_title("Rolling Hurst: Multiscale (>0.5 = trending)")
axes[1].legend()

plt.tight_layout()
plt.show()

## 4. Compute LPPLS confidence at each scale

In [ ]:
lppls_results = {}

# LPPLS window sizes per timeframe
lppls_nw_map = {"daily": 30, "3-day": 20, "weekly": 15}

for name, frame in timeframes.items():
    lp = log_prices(frame)
    t = time_index(frame)
    n_windows = lppls_nw_map[name]
    conf, tc_mean, tc_std = compute_confidence(
        t, lp, n_windows=n_windows, n_candidates=20,
    )
    conf = np.asarray(conf)
    lppls_results[name] = pd.Series(
        conf[:len(frame)],
        index=frame.index[:len(conf)],
        name=f"lppls_{name}",
    )
    valid = conf[~np.isnan(conf)]
    print(f"{name}: max LPPLS confidence = {valid.max():.3f}" if len(valid) > 0 else f"{name}: no valid")

In [ ]:
fig, axes = plt.subplots(2, 1, figsize=(14, 8), sharex=True)

axes[0].plot(df.index, df["close"], color="steelblue", linewidth=0.8)
axes[0].set_yscale("log")
axes[0].set_ylabel("Price (USD)")
axes[0].set_title("BTC/USD Price")

for name, series in lppls_results.items():
    axes[1].plot(series.index, series.values, color=colors[name],
                 linewidth=0.8, alpha=0.7, label=name)

axes[1].axhline(0.5, color="red", linestyle="--", alpha=0.5)
axes[1].set_ylabel("LPPLS Confidence")
axes[1].set_title("LPPLS Confidence: Multiscale")
axes[1].set_ylim(0, 1)
axes[1].legend()

plt.tight_layout()
plt.show()

## 5. Multiscale agreement

When multiple timeframes simultaneously show elevated risk signals, the probability
of a genuine crash is higher than any single-scale signal alone.

We compute agreement as the fraction of indicators (Hill < 3, DFA > 0.55,
LPPLS > 0.3) that are elevated across all three timeframes.

In [ ]:
# Resample all signals to weekly for alignment
weekly_idx = df_weekly.index

def resample_to_weekly(series):
    return series.resample("W").last().reindex(weekly_idx)

agreement_parts = []

# Hill: alpha < 3 = fat tails elevated
for name, series in hill_results.items():
    w = resample_to_weekly(series)
    agreement_parts.append((w < 3).astype(float))

# DFA: alpha > 0.55 = persistent
for name, series in dfa_results.items():
    w = resample_to_weekly(series)
    agreement_parts.append((w > 0.55).astype(float))

# LPPLS: confidence > 0.3
for name, series in lppls_results.items():
    w = resample_to_weekly(series)
    agreement_parts.append((w > 0.3).astype(float))

# Agreement = fraction of indicators that are elevated
stacked = pd.concat(agreement_parts, axis=1)
agreement = stacked.mean(axis=1)

print(f"Multiscale agreement range: [{agreement.min():.3f}, {agreement.max():.3f}]")
print(f"Weeks with agreement > 0.5: {(agreement > 0.5).sum()}/{len(agreement)}")

In [ ]:
fig, axes = plt.subplots(2, 1, figsize=(14, 8), sharex=True)

axes[0].plot(df.index, df["close"], color="steelblue", linewidth=0.8)
axes[0].set_yscale("log")
axes[0].set_ylabel("Price (USD)")
axes[0].set_title("BTC/USD Price")

axes[1].fill_between(
    agreement.index, 0, agreement,
    where=agreement > 0.5,
    color="red", alpha=0.4, label="High agreement (> 0.5)",
)
axes[1].fill_between(
    agreement.index, 0, agreement,
    where=agreement <= 0.5,
    color="gray", alpha=0.3, label="Low agreement",
)
axes[1].set_ylabel("Agreement Score")
axes[1].set_title("Multiscale Signal Agreement")
axes[1].set_ylim(0, 1)
axes[1].legend()

plt.tight_layout()
plt.show()

## 6. Cross-scale comparison table

Align all indicators on a common weekly grid and inspect periods of high agreement.

In [ ]:
comparison = pd.DataFrame(index=weekly_idx)

for name, series in hill_results.items():
    comparison[f"hill_{name}"] = resample_to_weekly(series)

for name, series in dfa_results.items():
    comparison[f"dfa_{name}"] = resample_to_weekly(series)

for name, series in lppls_results.items():
    comparison[f"lppls_{name}"] = resample_to_weekly(series)

comparison["agreement"] = agreement
comparison = comparison.dropna(how="all")

# Show periods where agreement is highest
high_alert = comparison[comparison["agreement"] > 0.5].sort_values("agreement", ascending=False)
print(f"Weeks with multi-timeframe alert: {len(high_alert)}")
print()
high_alert.head(15)

## Summary

- Indicators computed at multiple scales (daily, 3-day, weekly) provide independent
  views of crash risk.
- **Cross-scale agreement** is a powerful filter: when all scales show elevated risk,
  the signal is much more reliable than any single timeframe.
- Hill alpha dropping below 3 at all scales = distributional regime shift.
- DFA > 0.55 at all scales = persistent dynamics across timeframes.
- LPPLS confidence > 0.3 at all scales = bubble structure at every resolution.
- This multiscale agreement feeds into the composite crash signal (notebook 06).

*All numbers are in-sample on historical data. This is not financial advice.*